In [ ]:
import sys
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import timedelta
import time
from datetime import datetime
import redis
import struct
#from deepface import DeepFace
#from deepface.basemodels import Facenet
#from deepface.detectors import FaceDetector
#from deepface.commons import functions
import sys
sys.path.append('/opt/workspace/src/python_scripts/')
import ops_files_operations as files
import ops_logger as log
import ops_face_detection as fd
import ops_face_recognition as fr

In [ ]:
script_name = os.path.basename(__name__)
path_arr = os.getcwd().split(os.path.sep)
os.chdir(os.path.sep.join(path_arr[:path_arr.index('src')+1]))

In [ ]:
r = redis.StrictRedis(host="redis",port= 6379,db=0)
r.ping()

In [ ]:
def toRedis(r,a,n):
   """Store given Numpy array 'a' in Redis under key 'n'"""
   h, w = a.shape
   shape = struct.pack('>II',h,w)
   encoded = shape + a.tobytes()

   # Store encoded data in Redis
   r.set(n,encoded)
   return

def fromRedis(r,n):
   """Retrieve Numpy array from Redis key 'n'"""
   encoded = r.get(n)
   h, w = struct.unpack('>II',encoded[:8])
   # Add slicing here, or else the array would differ from the original
   a = np.frombuffer(encoded[8:],dtype=np.uint16).reshape(h,w)
   return a

# Create 80x80 numpy array to store
#a0 = np.arange(6400,dtype=np.uint16).reshape(80,80) 

# Redis connection
#r = redis.Redis(host='localhost', port=6379, db=0)

# Store array a0 in Redis under name 'a0array'
#toRedis(r,a0,'a0array')

# Retrieve from Redis
#a1 = fromRedis(r,'a0array')

#np.testing.assert_array_equal(a0,a1 )

### Face detection

In [ ]:
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']

#face detection and alignment
detected_face = DeepFace.detectFace(test_image, target_size = (160, 160) ,detector_backend = backends[0])

##face verification
#obj = DeepFace.verify("img1.jpg", "img2.jpg", detector_backend = backends[4])

#face recognition
#df = DeepFace.find(img_path = "img.jpg", db_path = "my_db", detector_backend = backends[4])

#facial analysis
#demography = DeepFace.analyze("img4.jpg", detector_backend = backends[4])

In [ ]:
img = cv2.imread(two_person)
face = get_faces_from_img(img)

In [ ]:

logger = log.create_logger(level = 'info')
starttime = time.time()
cap = cv2.VideoCapture(video_file)
video_fps = cap.get(cv2.CAP_PROP_FPS)
desired_fps = 1
results = []
frame_no = 0
frames_read = 0

while(cap.isOpened()):
    frame_exists, curr_frame = cap.read()
    if frame_exists:
        if frame_no % (round(video_fps/desired_fps)) == 0:
            try:
                frames_read+=1
                #frame_timestamp = timedelta(milliseconds = cap.get(cv2.CAP_PROP_POS_MSEC))
                frame_timestamp = time.strftime('%H:%M:%S.{}'.format(round(round((frame_no/video_fps) % 1,3)*1000)), time.gmtime(frame_no/video_fps))
                frame_embeddings =  np.array(fd.get_embeddings_from_image(provided_image =curr_frame, multiple_faces = True, logger = logger ))
                results.append([str(frame_no),str(frame_timestamp),frame_embeddings])
                
            except:
                logger.error("An error ocurred:",sys.exc_info()[0])

    else:
        break

    frame_no += 1

    
results_array = np.asarray(results, dtype=object)
endtime = time.time()
exec_time = str(endtime - starttime)
logger.info(f'total execution time = {exec_time} seconds. Total frames: {frames_read}')
log.shutdown_logger(logger)

cap.release()
cv2.destroyAllWindows()

In [ ]:
video_file = './datasets/videos/the_final_kick.mp4'
results,dict = fd.process_video(video_file)

In [ ]:
results

In [ ]:
probs = fr.predict_probabilities([results_array[11][2][0]],'./models/embeddings','./models/recognizers')

In [ ]:
video_file = './datasets/videos/the_final_kick.mp4'
results_path = './models/embeddings/processed_videos'
fd.get_video_embeddings(video_file,results_path,partitions = 2)

In [ ]:
frame = 3984
frame_timestamp = time.strftime('%H:%M:%S.{}'.format(round(round((frame/video_fps) % 1,3)*1000)), time.gmtime(frame/video_fps))

In [ ]:
print(results)

In [ ]:
print(24%24)

In [ ]:
#2614-2637 01:47

In [ ]:
image_path = './datasets/actor_faces/2037_cillian_murphy/cillian_murphy_1.jpg'
#embeddings, metadata = fd.get_embeddings_from_image(image_path)
dataset_folder = './datasets/actor_faces/'
embeddings, metadata = fd.get_actors_embeddings(dataset_folder)

In [ ]:
model_dict = {
        "model_name": "test1",
        "embeddings_path": "my/path/lol"
    }
model_dict

In [ ]:
model_dict.update(metadata)

In [ ]:
embeddings_index_file = './models/embeddings/actor_faces/embeddings_metadata.json'
files.add_embeddings_data(embeddings_index_file,model_dict)

In [ ]:
embeddings_array=[]
embeddings_array.append(model_dict)

In [ ]:
embeddings_array

In [ ]:
import json
with open(embeddings_index_file,'w') as json_file:
    json.dump(embeddings_array_test, json_file, indent=4, separators=(',',': '))

### Capturing one frame of video

In [ ]:
frame_number = 1
cap = cv2.VideoCapture(video_file)
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
frame_exists, frame = cap.read()

if frame_exists:
    #plt.imshow(frame)
    face_object = get_faces_from_img(frame)
    embeddings = get_embedding_for_img(face_object[0][0][:,:,::-1])
    print(embeddings)
    plt.imshow(face_object[0][0][:,:,::-1])
    print(len(face_object))
    

cap.release()
cv2.destroyAllWindows()

In [ ]:
frame = get_frame(video_file,2639)
print('shape is', frame.shape)
print('pixel at (60,21)',frame[60,21,:])
print('pixel at (120,10)',frame[120,10,:])
plt.imshow(frame)

In [ ]:
fix_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
print('pixel at (120,10)',fix_frame[120,10,:])
plt.imshow(fix_frame)

In [ ]:
frames = 3647
partitions = 7
chunks = []
chunk_size = round(frames/partitions)
for i in range(partitions):
    starting_frame = i*chunk_size
    ending_frame = (starting_frame + chunk_size) - 1
    if ending_frame > frames or partitions == 1:
        ending_frame = frames
    chunk = [starting_frame,ending_frame]
    chunks.append(chunk)
chunks

In [ ]:
for idx,chunk in enumerate(chunks):
    
    print(chunk)

In [ ]:
video_path = './datasets/videos/dream_scene.mp4'
file_name = video_path.split(os.path.sep)[-1].split('.')[0] + '_1'
file

In [ ]:
path = './models/embeddings/processed_videos/the_final_kick/the_final_kick_1.pickle'
os.path.dirname(path)

In [ ]:
metadict = {
    "test": "hi",
    "test_2": "nope",
    "chunks": []
}

dict1 = {
    "color": "red",
    "size": 1
}

dict2 = {
    "color": "blue",
    "size": 2
}

In [ ]:
array_dict = [dict1,dict2]

In [ ]:
array_dict[1]

In [ ]:
metadict["chunks"].append(dict2)

In [ ]:
metadict

In [ ]:
import uuid

In [ ]:
str(uuid.uuid4())[0:8]